## lyrics

start holiday embrace mariana climate besiege tied commendable slight manners logic bastards parents toads pilate ten gains denying george withdraw spices muse abbot painting galled heel beggarly proves canon acknowledge harvest tapster avoided aqua gyves dozen gallops wealthy provide offences hair pence thoroughly bustle lean forbidden parlous mock seek smells thinking petition shun method madly female wrinkle blessed rare fondly

In [21]:

# 드라이브 마운트

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import glob #glob 모듈의 glob 함수는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환한다
import os

lyrics_file_path = '/content/drive/MyDrive/Colab Notebooks/Data/shakespeare.txt'

txt_list = glob.glob(lyrics_file_path) #txt_file_path 경로에 있는 모든 파일명을 리스트 형식으로 txt_list 에 할당

In [2]:
raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines() #read() : 파일 전체의 내용을 하나의 문자열로 읽어온다. , splitlines()  : 여러라인으로 구분되어 있는 문자열을 한라인씩 분리하여 리스트로 반환
        raw_corpus.extend(raw) # extend() : 리스트함수로 추가적인 내용을 연장 한다.

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])


데이터 크기: 40000
Examples:
 ['First Citizen:', 'Before we proceed any further, hear me speak.', '']


In [3]:
import re

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = ' ' + sentence + ' ' # 6
    return sentence

corpus = list(map(preprocess_sentence, raw_corpus))

In [48]:
import numpy as np
import tensorflow as tf

# 토크나이저 함수로 Tensor 변환

def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=20000,
        filters=' ',
        oov_token=""
    )

    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen=15)

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  98  281    0 ...    0    0    0]
 [ 150   38  982 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 ...
 [ 166  582    2 ...    0    0    0]
 [  31   69  145 ...    0    0    0]
 [1074   31  141 ...    0    0    0]] <keras.src.legacy.preprocessing.text.Tokenizer object at 0x7ba4e1044740>


In [49]:
from sklearn.model_selection import train_test_split

enc_inputs = tensor[:, :-1]
dec_targets = tensor[:, 1:]

# 20%를 평가 데이터로 분리
enc_train, enc_val, dec_train, dec_val = train_test_split(
    enc_inputs,
    dec_targets,
    test_size=0.2,
    random_state=42,  # 재현성 확보 위해 시드 고정 (선택 사항)
    shuffle=True      # 데이터 섞기
)

In [50]:
enc_train

array([[480,   6,  96, ...,   0,   0,   0],
       [ 66,  61,  23, ...,   0,   0,   0],
       [101,   5,  33, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,   0,   0,   0],
       [ 98, 691,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]], dtype=int32)

In [51]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 128
steps_per_epoch = len(enc_train) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

val_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
val_dataset = val_dataset.shuffle(BUFFER_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)
print(val_dataset)

<_BatchDataset element_spec=(TensorSpec(shape=(128, 14), dtype=tf.int32, name=None), TensorSpec(shape=(128, 14), dtype=tf.int32, name=None))>
<_BatchDataset element_spec=(TensorSpec(shape=(128, 14), dtype=tf.int32, name=None), TensorSpec(shape=(128, 14), dtype=tf.int32, name=None))>


In [52]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        # Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성되어 있다.
        # Embedding 레이어는 단어 사전의 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔준다.
        # 이 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현으로 사용된다.
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)

        return out
# embedding size 값이 커질수록 단어의 추상적인 특징들을 더 잡아낼 수 있지만
# 그만큼 충분한 데이터가 없으면 안좋은 결과 값을 가져옵니다!
embedding_size = 256 # 워드 벡터의 차원수를 말하며 단어가 추상적으로 표현되는 크기입니다.
hidden_size = 1024 # 모델에 얼마나 많은 일꾼을 둘 것인가? 정도로 이해하면 좋다.
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size) # tokenizer.num_words에 +1인 이유는 문장에 없는 pad가 사용되었기 때문이다.

In [61]:
def generate_text(model, tokenizer, init_sentence="", max_len=30, temperature=1.0):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)

    # Check if '' is in word_index, if not, handle it (e.g., set a default or skip)
    end_token = tokenizer.word_index.get("", None)

    while True:
        # 1. 모델 예측 (logits)
        predict = model(test_tensor)
        # 2. Temperature를 적용하여 예측된 확률 분포 조절
        predict = predict / temperature
        # 3. Softmax를 적용하여 확률 분포 얻기
        predict_word_probabilities = tf.nn.softmax(predict, axis=-1)
        # 4. 확률 분포에서 다음 단어 샘플링
        predict_word = tf.random.categorical(predict_word_probabilities[:, -1], num_samples=1)[0].numpy()
        predict_word = predict_word[0] # tf.random.categorical returns a 2D tensor, get the scalar

        # 5. 샘플링된 단어 인덱스를 텐서로 확장
        test_tensor = tf.concat([test_tensor, tf.expand_dims(tf.constant([predict_word], dtype=tf.int64), axis=0)], axis=-1)

        # 6. 종료 조건 확인
        if end_token is not None and predict_word == end_token:
            break
        if test_tensor.shape[1] >= max_len:
            break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다
    for word_index in test_tensor[0].numpy():
        if word_index in tokenizer.index_word:
            generated += tokenizer.index_word[word_index] + " "
        else:
            generated += " "

    return generated.strip()


generate_text function updated to include temperature sampling and fixed concat rank mismatch.


In [62]:
generate_text(model, tokenizer, init_sentence=" start ", max_len=60)

'start holiday embrace mariana climate besiege tied commendable slight manners logic bastards parents toads pilate ten gains denying george withdraw spices muse abbot painting galled heel beggarly proves canon acknowledge harvest tapster avoided aqua gyves dozen gallops wealthy provide offences hair pence thoroughly bustle lean forbidden parlous mock seek smells thinking petition shun method madly female wrinkle blessed rare fondly'